# Setup

### Import Dependencies

In [1]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup as bs
import re
import requests
import time
import pandas as pd
import unicodedata
import random

### Set URLs

In [2]:
# base URL for race results
base_url = "https://results.nyrr.org/event/"

# specific races

race_code = "M2019" # 2019 NYC Marathon. includes gun time and splits

# race_code = "20WH5K" # 2020 Washington Heights 5K. 4942 finishers. yes to gun time, no to splits

# race_code = "a70414" # 2007 Brooklyn Half. 4853 finishers. no gun time, no splits

# race_code = "19nyc60" # 2019 NYC 60K. 375 finishers, gun time, no splits

# race_code = "1860k" # 2018 NYC 60K. 360 finishers, gun time, no splits

# race_code = "1760K" # 2017 NYC 60K. 358 finishers (plus one handcycle), gun time, no splits

# race_code = "60K16" # 2016 NYC 60K. 366 finishers, gun time, no splits. 2 runners do not have an address.

race_url = base_url + race_code + "/result/"

### Set File Locations

In [3]:
# Set path for chromedriver
executable_path = {'chromedriver.exe'}

# set paths for outputs
output_csv_path = "../00_data/raw_data/"
output_csv_name_base = "finishers"

log_csv_path = "../00_data/raw_data/"
log_csv_name_base = "scraping_log"

### Set Scraping Parameters

In [4]:
# lowest bib number in scraping range
low_bib = 72001

# highest bib number in scraping range
high_bib = 75000

# add any additional targets
additional_scraping_targets = []


# Scraping

### Define scraping function

In [5]:
def finisher_scraper(soup):
    finisher_dict = {}
# block 1
    thangs = soup.find(class_='cursor-pointer results-link').findChildren()
    finisher_dict['name'] = thangs[0].text
    
    # The address is presented as a subregion, then a pipe "|", then a country code.
    # For US addresses, the subregion is presented as town followed by state abbreviation.
    # Some runners don't have a full address listed, so carefully parse out what is there.
    # Assume that the most basic information would be the runner's country (i.e. country of residence).
    # Note that these are ISO 3166-1 country codes, e.g. Germany = 'DEU'.
    # These items do not have labels (e.g. 'country') associated with them, so
    # the keys are hardcoded here.
    
    if '|' in thangs[1].text:
        finisher_dict['geo_subregion'] = thangs[1].text.split('|')[0].strip()
        finisher_dict['country'] = thangs[1].text.split('|')[1].strip()
    else:
        finisher_dict['country'] = thangs[1].text.strip()
    
    for thing in soup.find_all('strong'):
        if thing.next_sibling:
            
            #look for gender and age, and record those separately.
            
            pattern = re.compile('[MF][1-9][0-9]?')
            if pattern.match(thing.next_sibling.strip()):
                finisher_dict['gender'] = thing.next_sibling.strip()[0]
                finisher_dict['age'] = thing.next_sibling.strip()[1:]
                continue
                
            # I prefer the keys and columns be lowercase without spaces, so make those changes
            # while reading in the keys.
            
            finisher_dict[thing.text.strip().lower().replace(" ", "_").replace("/", "_and_")] = thing.next_sibling.strip()
            
# block 2 and block 3
    regex = re.compile('.*form-group-item.*')
#     block_2_soup = soup.find_all("div", {"class" : regex})[:4] # this gets only block 2
    block_2_soup = soup.find_all("div", {"class" : regex}) # this gets block 3 as well
    for spoonful in block_2_soup:
        
        # The data found here can vary from race to race and runner to runner. These items have labels
        # associated with them. To be flexible, we read the label itself and use it as the dict key.
        # Not every runner will have a full set of label-value pairs, and the pair vary from race to race.
        # For example, a runner's bib might not get recorded at a particular checkpoint, and a half marathon
        # would not have as many checkpoints as a marathon.
        
        labels = spoonful.find_all('label')
        values = spoonful.find_all('span', {'class': 'label-value ng-binding'})
        
        # Step through all the labels and add each key(label)-value pair to the dict.
        
        for x in range(0, len(labels)):
            
            # there was a '\xa0' coming in as part of the 'Place Country' label
            # use unicodedata.normalize to replace that with a space.
            
            label = unicodedata.normalize("NFKD", labels[x].text)
            
            # if we find the place in age-group, add the age group to the dictionary.
            
            if label.endswith("Place Age‐Group"):
                finisher_dict["age_group"] = re.findall(r'\(([^]]*)\)', label)[0]
                label = "Place Age-Group"
                
            # If we find the place in country, add the country group (i.e. country of citizenship) to the dictionary.
            # Note that this is not necessarily the same as the runner's address.
            # This country code is in IOC format, e.g. Germany = 'GER'.
                
            elif label.endswith("Place Country"):
                finisher_dict["country_group"] = re.findall(r'\(([^]]*)\)', label)[0]
                label = "Place Country"
                
            # Add pairs to dict. Make sure keys are lowecase without spaces.
            
            finisher_dict[label.strip().lower().replace(" ", "_")] = values[x].text


    return finisher_dict
    

In [6]:
def scraping_loop(race_url, scrape_list):
    
    # These lists will be returned.
    finishers = []
    dnf_list = []
    scraping_log = []
    error_list = []

    # These counters are used for reporting progress.
    attempt_counter = 0
    hit_counter = 0
    miss_counter = 0
    error_counter = 0
    
    browser = Browser('chrome', headless=False)

    for bib_number in scrape_list:
        url = race_url + str(bib_number)
        browser.visit(url)
        # We need a short sleep period to give the site a chance to redirect an invalid bib number.
        # We also don't need to beat on the website too hard.
        time.sleep(3)
    #     time.sleep(random.uniform(3,7))
        attempt_counter += 1
        
        # If the url does not redirect, then the bib number crossed the finish line. Scrape it.
        if browser.url == url:
            try:
                soup = bs(browser.html, "html.parser")
                new_result = finisher_scraper(soup)

                # We're being intentionally flexible about what data we gather, 
                # but bib number and finish time are the bare minimum.
                # Check to see if there is a bib number and official finish time included in the finisher_dict.
                if (new_result.get('bib') and new_result.get('name')):
                    finishers.append(new_result)
                    scraping_log.append({'bib': bib_number, 'outcome': 'success'})
                    hit_counter += 1
                    
                # If the url was good, but bib number and official time are missing, then something went wrong during
                # scraping. Log it as an error.
                else:
                    scraping_log.append({'bib': bib_number, 'outcome': 'error'})
                    error_list.append(bib_number)
                    error_counter += 1

            # If bs can't even parse the page, something went wrong. Log it as an error.
            except:
                scraping_log.append({'bib': bib_number, 'outcome': 'error'})
                error_list.append(bib_number)
                error_counter += 1
        
        # If the url resulted in a redirect, then that bib number never crossed the finish line. Log it as DNF.
        # Note that the bib may never have been issued, or the runner may have dropped out 50m from the finish.
        # All we know is that the bib was not logged as a finisher.
        else:
            scraping_log.append({'bib': bib_number, 'outcome': 'DNF'})
            dnf_list.append(bib_number)
            miss_counter += 1
        
        # Report running totals of the scraping progress.
        print(f"Attempted {attempt_counter} of {len(scrape_list)}: Found {hit_counter} finishers ({(hit_counter/attempt_counter):.2%}), {miss_counter} DNFs, and {error_counter} errors.")
    
    print('Loop complete')
    print(f"{attempt_counter} attempts yielded {hit_counter} hits ({(hit_counter/attempt_counter):.2%}) with {error_counter} errors.")
    browser.quit()
    
    return [finishers, dnf_list, error_list, scraping_log]

### Scraping Loop

In [7]:
# Build the list of bibs to scrape.
scrape_list = list(range(low_bib,high_bib+1))

# Add in any additional scraping targets, if applicable.
scrape_list.extend(additional_scraping_targets)
    
# Shuffle the list. This helps us to get an ide of the 'richness' of the specified bib range is after a few dozen scrapes.
random.shuffle(scrape_list)

In [8]:
# scrape through the list
scrape_results = scraping_loop(race_url, scrape_list)
runners = scrape_results[0]
scraping_log = scrape_results[3]

# Go back and retry any bibs that didn't load on the first try.
if scrape_results[2]:
    error_list = scrape_results[2]
    second_scrape = scraping_loop(race_url, error_list)
    runners.extend(second_scrape[0])
    scraping_log.extend(second_scrape[3])

Attempted 1 of 3000: Found 1 finishers (100.00%), 0 DNFs, and 0 errors.
Attempted 2 of 3000: Found 1 finishers (50.00%), 1 DNFs, and 0 errors.
Attempted 3 of 3000: Found 2 finishers (66.67%), 1 DNFs, and 0 errors.
Attempted 4 of 3000: Found 2 finishers (50.00%), 2 DNFs, and 0 errors.
Attempted 5 of 3000: Found 2 finishers (40.00%), 3 DNFs, and 0 errors.
Attempted 6 of 3000: Found 2 finishers (33.33%), 4 DNFs, and 0 errors.
Attempted 7 of 3000: Found 2 finishers (28.57%), 5 DNFs, and 0 errors.
Attempted 8 of 3000: Found 3 finishers (37.50%), 5 DNFs, and 0 errors.
Attempted 9 of 3000: Found 3 finishers (33.33%), 6 DNFs, and 0 errors.
Attempted 10 of 3000: Found 3 finishers (30.00%), 7 DNFs, and 0 errors.
Attempted 11 of 3000: Found 3 finishers (27.27%), 8 DNFs, and 0 errors.
Attempted 12 of 3000: Found 3 finishers (25.00%), 9 DNFs, and 0 errors.
Attempted 13 of 3000: Found 4 finishers (30.77%), 9 DNFs, and 0 errors.
Attempted 14 of 3000: Found 4 finishers (28.57%), 10 DNFs, and 0 errors.

Attempted 113 of 3000: Found 19 finishers (16.81%), 94 DNFs, and 0 errors.
Attempted 114 of 3000: Found 19 finishers (16.67%), 95 DNFs, and 0 errors.
Attempted 115 of 3000: Found 19 finishers (16.52%), 96 DNFs, and 0 errors.
Attempted 116 of 3000: Found 19 finishers (16.38%), 97 DNFs, and 0 errors.
Attempted 117 of 3000: Found 20 finishers (17.09%), 97 DNFs, and 0 errors.
Attempted 118 of 3000: Found 20 finishers (16.95%), 98 DNFs, and 0 errors.
Attempted 119 of 3000: Found 21 finishers (17.65%), 98 DNFs, and 0 errors.
Attempted 120 of 3000: Found 21 finishers (17.50%), 99 DNFs, and 0 errors.
Attempted 121 of 3000: Found 21 finishers (17.36%), 100 DNFs, and 0 errors.
Attempted 122 of 3000: Found 21 finishers (17.21%), 101 DNFs, and 0 errors.
Attempted 123 of 3000: Found 21 finishers (17.07%), 102 DNFs, and 0 errors.
Attempted 124 of 3000: Found 22 finishers (17.74%), 102 DNFs, and 0 errors.
Attempted 125 of 3000: Found 22 finishers (17.60%), 103 DNFs, and 0 errors.
Attempted 126 of 300

Attempted 221 of 3000: Found 43 finishers (19.46%), 178 DNFs, and 0 errors.
Attempted 222 of 3000: Found 44 finishers (19.82%), 178 DNFs, and 0 errors.
Attempted 223 of 3000: Found 44 finishers (19.73%), 179 DNFs, and 0 errors.
Attempted 224 of 3000: Found 44 finishers (19.64%), 180 DNFs, and 0 errors.
Attempted 225 of 3000: Found 44 finishers (19.56%), 181 DNFs, and 0 errors.
Attempted 226 of 3000: Found 44 finishers (19.47%), 182 DNFs, and 0 errors.
Attempted 227 of 3000: Found 45 finishers (19.82%), 182 DNFs, and 0 errors.
Attempted 228 of 3000: Found 45 finishers (19.74%), 183 DNFs, and 0 errors.
Attempted 229 of 3000: Found 45 finishers (19.65%), 184 DNFs, and 0 errors.
Attempted 230 of 3000: Found 45 finishers (19.57%), 185 DNFs, and 0 errors.
Attempted 231 of 3000: Found 45 finishers (19.48%), 186 DNFs, and 0 errors.
Attempted 232 of 3000: Found 45 finishers (19.40%), 187 DNFs, and 0 errors.
Attempted 233 of 3000: Found 45 finishers (19.31%), 188 DNFs, and 0 errors.
Attempted 23

Attempted 329 of 3000: Found 61 finishers (18.54%), 268 DNFs, and 0 errors.
Attempted 330 of 3000: Found 61 finishers (18.48%), 269 DNFs, and 0 errors.
Attempted 331 of 3000: Found 61 finishers (18.43%), 270 DNFs, and 0 errors.
Attempted 332 of 3000: Found 61 finishers (18.37%), 271 DNFs, and 0 errors.
Attempted 333 of 3000: Found 61 finishers (18.32%), 272 DNFs, and 0 errors.
Attempted 334 of 3000: Found 61 finishers (18.26%), 273 DNFs, and 0 errors.
Attempted 335 of 3000: Found 61 finishers (18.21%), 274 DNFs, and 0 errors.
Attempted 336 of 3000: Found 62 finishers (18.45%), 274 DNFs, and 0 errors.
Attempted 337 of 3000: Found 62 finishers (18.40%), 275 DNFs, and 0 errors.
Attempted 338 of 3000: Found 62 finishers (18.34%), 276 DNFs, and 0 errors.
Attempted 339 of 3000: Found 62 finishers (18.29%), 277 DNFs, and 0 errors.
Attempted 340 of 3000: Found 62 finishers (18.24%), 278 DNFs, and 0 errors.
Attempted 341 of 3000: Found 62 finishers (18.18%), 279 DNFs, and 0 errors.
Attempted 34

Attempted 437 of 3000: Found 80 finishers (18.31%), 357 DNFs, and 0 errors.
Attempted 438 of 3000: Found 81 finishers (18.49%), 357 DNFs, and 0 errors.
Attempted 439 of 3000: Found 81 finishers (18.45%), 358 DNFs, and 0 errors.
Attempted 440 of 3000: Found 81 finishers (18.41%), 359 DNFs, and 0 errors.
Attempted 441 of 3000: Found 81 finishers (18.37%), 360 DNFs, and 0 errors.
Attempted 442 of 3000: Found 81 finishers (18.33%), 361 DNFs, and 0 errors.
Attempted 443 of 3000: Found 81 finishers (18.28%), 362 DNFs, and 0 errors.
Attempted 444 of 3000: Found 81 finishers (18.24%), 363 DNFs, and 0 errors.
Attempted 445 of 3000: Found 81 finishers (18.20%), 364 DNFs, and 0 errors.
Attempted 446 of 3000: Found 81 finishers (18.16%), 365 DNFs, and 0 errors.
Attempted 447 of 3000: Found 82 finishers (18.34%), 365 DNFs, and 0 errors.
Attempted 448 of 3000: Found 82 finishers (18.30%), 366 DNFs, and 0 errors.
Attempted 449 of 3000: Found 82 finishers (18.26%), 367 DNFs, and 0 errors.
Attempted 45

...

Attempted 2538 of 3000: Found 480 finishers (18.91%), 2057 DNFs, and 1 errors.
Attempted 2539 of 3000: Found 480 finishers (18.91%), 2058 DNFs, and 1 errors.
Attempted 2540 of 3000: Found 480 finishers (18.90%), 2059 DNFs, and 1 errors.
Attempted 2541 of 3000: Found 481 finishers (18.93%), 2059 DNFs, and 1 errors.
Attempted 2542 of 3000: Found 481 finishers (18.92%), 2060 DNFs, and 1 errors.
Attempted 2543 of 3000: Found 482 finishers (18.95%), 2060 DNFs, and 1 errors.
Attempted 2544 of 3000: Found 482 finishers (18.95%), 2061 DNFs, and 1 errors.
Attempted 2545 of 3000: Found 482 finishers (18.94%), 2062 DNFs, and 1 errors.
Attempted 2546 of 3000: Found 482 finishers (18.93%), 2063 DNFs, and 1 errors.
Attempted 2547 of 3000: Found 483 finishers (18.96%), 2063 DNFs, and 1 errors.
Attempted 2548 of 3000: Found 483 finishers (18.96%), 2064 DNFs, and 1 errors.
Attempted 2549 of 3000: Found 484 finishers (18.99%), 2064 DNFs, and 1 errors.
Attempted 2550 of 3000: Found 485 finishers (19.02%)

Attempted 2642 of 3000: Found 511 finishers (19.34%), 2130 DNFs, and 1 errors.
Attempted 2643 of 3000: Found 511 finishers (19.33%), 2131 DNFs, and 1 errors.
Attempted 2644 of 3000: Found 511 finishers (19.33%), 2132 DNFs, and 1 errors.
Attempted 2645 of 3000: Found 511 finishers (19.32%), 2133 DNFs, and 1 errors.
Attempted 2646 of 3000: Found 511 finishers (19.31%), 2134 DNFs, and 1 errors.
Attempted 2647 of 3000: Found 511 finishers (19.30%), 2135 DNFs, and 1 errors.
Attempted 2648 of 3000: Found 511 finishers (19.30%), 2136 DNFs, and 1 errors.
Attempted 2649 of 3000: Found 511 finishers (19.29%), 2137 DNFs, and 1 errors.
Attempted 2650 of 3000: Found 511 finishers (19.28%), 2138 DNFs, and 1 errors.
Attempted 2651 of 3000: Found 511 finishers (19.28%), 2139 DNFs, and 1 errors.
Attempted 2652 of 3000: Found 511 finishers (19.27%), 2140 DNFs, and 1 errors.
Attempted 2653 of 3000: Found 512 finishers (19.30%), 2140 DNFs, and 1 errors.
Attempted 2654 of 3000: Found 513 finishers (19.33%)

Attempted 2746 of 3000: Found 533 finishers (19.41%), 2211 DNFs, and 2 errors.
Attempted 2747 of 3000: Found 534 finishers (19.44%), 2211 DNFs, and 2 errors.
Attempted 2748 of 3000: Found 534 finishers (19.43%), 2212 DNFs, and 2 errors.
Attempted 2749 of 3000: Found 534 finishers (19.43%), 2213 DNFs, and 2 errors.
Attempted 2750 of 3000: Found 534 finishers (19.42%), 2214 DNFs, and 2 errors.
Attempted 2751 of 3000: Found 534 finishers (19.41%), 2215 DNFs, and 2 errors.
Attempted 2752 of 3000: Found 534 finishers (19.40%), 2216 DNFs, and 2 errors.
Attempted 2753 of 3000: Found 534 finishers (19.40%), 2217 DNFs, and 2 errors.
Attempted 2754 of 3000: Found 534 finishers (19.39%), 2218 DNFs, and 2 errors.
Attempted 2755 of 3000: Found 535 finishers (19.42%), 2218 DNFs, and 2 errors.
Attempted 2756 of 3000: Found 536 finishers (19.45%), 2218 DNFs, and 2 errors.
Attempted 2757 of 3000: Found 536 finishers (19.44%), 2219 DNFs, and 2 errors.
Attempted 2758 of 3000: Found 536 finishers (19.43%)

Attempted 2850 of 3000: Found 559 finishers (19.61%), 2289 DNFs, and 2 errors.
Attempted 2851 of 3000: Found 559 finishers (19.61%), 2290 DNFs, and 2 errors.
Attempted 2852 of 3000: Found 559 finishers (19.60%), 2291 DNFs, and 2 errors.
Attempted 2853 of 3000: Found 559 finishers (19.59%), 2292 DNFs, and 2 errors.
Attempted 2854 of 3000: Found 560 finishers (19.62%), 2292 DNFs, and 2 errors.
Attempted 2855 of 3000: Found 560 finishers (19.61%), 2293 DNFs, and 2 errors.
Attempted 2856 of 3000: Found 560 finishers (19.61%), 2294 DNFs, and 2 errors.
Attempted 2857 of 3000: Found 560 finishers (19.60%), 2295 DNFs, and 2 errors.
Attempted 2858 of 3000: Found 560 finishers (19.59%), 2296 DNFs, and 2 errors.
Attempted 2859 of 3000: Found 560 finishers (19.59%), 2297 DNFs, and 2 errors.
Attempted 2860 of 3000: Found 561 finishers (19.62%), 2297 DNFs, and 2 errors.
Attempted 2861 of 3000: Found 561 finishers (19.61%), 2298 DNFs, and 2 errors.
Attempted 2862 of 3000: Found 561 finishers (19.60%)

Attempted 2954 of 3000: Found 574 finishers (19.43%), 2378 DNFs, and 2 errors.
Attempted 2955 of 3000: Found 574 finishers (19.42%), 2379 DNFs, and 2 errors.
Attempted 2956 of 3000: Found 574 finishers (19.42%), 2380 DNFs, and 2 errors.
Attempted 2957 of 3000: Found 574 finishers (19.41%), 2381 DNFs, and 2 errors.
Attempted 2958 of 3000: Found 574 finishers (19.41%), 2382 DNFs, and 2 errors.
Attempted 2959 of 3000: Found 574 finishers (19.40%), 2383 DNFs, and 2 errors.
Attempted 2960 of 3000: Found 574 finishers (19.39%), 2384 DNFs, and 2 errors.
Attempted 2961 of 3000: Found 574 finishers (19.39%), 2385 DNFs, and 2 errors.
Attempted 2962 of 3000: Found 575 finishers (19.41%), 2385 DNFs, and 2 errors.
Attempted 2963 of 3000: Found 575 finishers (19.41%), 2386 DNFs, and 2 errors.
Attempted 2964 of 3000: Found 575 finishers (19.40%), 2387 DNFs, and 2 errors.
Attempted 2965 of 3000: Found 575 finishers (19.39%), 2388 DNFs, and 2 errors.
Attempted 2966 of 3000: Found 575 finishers (19.39%)

# EDA

### Convert scraped data to dataframe

In [9]:
finishers_df = pd.DataFrame(runners)

In [10]:
finishers_df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582 entries, 0 to 581
Data columns (total 54 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   582 non-null    object
 1   geo_subregion          582 non-null    object
 2   country                582 non-null    object
 3   gender                 582 non-null    object
 4   age                    582 non-null    object
 5   bib                    582 non-null    object
 6   official_time          582 non-null    object
 7   pace_per_mile          582 non-null    object
 8   place_overall          582 non-null    object
 9   place_gender           582 non-null    object
 10  age_group              582 non-null    object
 11  place_age-group        582 non-null    object
 12  country_group          582 non-null    object
 13  place_country          582 non-null    object
 14  place_age‐graded       582 non-null    object
 15  time_age‐graded        

In [11]:
finishers_df

,name,geo_subregion,country,gender,age,bib,official_time,pace_per_mile,place_overall,place_gender,...,21m,35k,22m,23m,24m,40k,25m,26m,mar,team
0,Dina Castellanos,Chetumal,MEX,F,61,72385,7:11:12,16:27,"52,661","22,182",...,5:41:49,5:54:05,5:58:18,6:14:50,6:32:44,6:47:24,6:50:01,7:07:24,7:11:12,NaN
1,Rajesh Sunder,"Lutz, FL",USA,M,44,72665,6:11:36,14:11,"49,879","29,372",...,4:53:51,5:05:12,5:08:59,5:24:18,5:39:41,5:52:38,5:54:40,6:08:52,6:11:36,NaN
2,Cynthia Marfori-Gordon,"New York, NY",USA,F,57,72073,7:19:00,16:45,"52,824","22,277",...,5:55:08,6:08:34,6:12:36,NaN,6:45:06,6:58:05,7:00:22,7:15:34,7:19:00,New York Walkers Club
3,Nichol Hare,"New York, NY",USA,F,40,72512,6:28:59,14:51,"51,068","21,248",...,5:10:56,5:22:28,5:25:55,5:42:28,5:57:23,6:09:21,6:11:45,6:25:51,6:28:59,NaN
4,Sarath Challa,"New Milford, NJ",USA,M,35,72444,7:30:37,17:12,"53,031","30,620",...,5:42:02,5:56:48,6:01:44,NaN,6:41:41,6:59:38,7:03:03,7:25:41,7:30:37,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,Iain Craig,Auckland,NZL,M,55,72531,5:56:45,13:37,"48,458","28,762",...,4:45:51,4:55:04,4:58:34,5:11:26,5:25:57,5:38:22,5:40:06,5:53:49,5:56:45,NaN
578,Danny Mota,"Brooklyn, NY",USA,M,41,72047,6:33:56,15:02,"51,339","29,948",...,5:05:45,5:17:30,5:21:47,5:39:31,5:59:04,6:12:02,6:14:46,6:31:02,6:33:56,NaN
579,Johnine Barnes,"Washington, DC",USA,F,49,72117,6:31:22,14:56,"51,203","21,318",...,5:05:54,5:18:12,5:22:37,5:40:30,5:58:01,6:11:13,6:13:33,6:28:13,6:31:22,Black Girls Run
580,Julie Romain,"Buffalo, NY",USA,F,50,72521,6:49:27,15:37,"51,998","21,759",...,5:20:40,5:33:33,5:37:56,5:55:01,6:12:19,6:26:58,6:29:28,6:46:02,6:49:27,NaN


# Store Data

### Write finishers to csv

In [12]:
def df_to_csv_writer(target_df, destination_file):
    import csv

    # We want to match the order of columns if we are appending new entries to an existing file.
    ### Because we are using a dictionary to capture the race results, and of the way dictionaries are ordered,
    ### It is possible for different scraping runs to attempt to order the columns differently.
    ### That would be confusing, so we force whatever order was established in the first run.
    ### (The bottom line is that it would really be a good idea to use a database here instead.)

    # Try to read the header from the existing file. If they exist, match the column order.
    try:
        with open(destination_file, 'r', encoding="utf-8") as f:
            reader = csv.reader(f)
            header_list = next(reader)
            # It is possible to be missing columns, so check to see if we have everything.
            for x in header_list:
                if x not in target_df.columns:
                    target_df[x] = ''
            target_df = target_df[header_list]
    # If we weren't able to open the file, we are going to create one from scratch.
    except:
        print(f"{destination_file} does not exist, so I'll create it!")

    # This either creates the file, or appends to it without headers. We already took steps to make sure
    # the columns in finishers_df match the headers (if they exist).
    with open(destination_file, 'a', encoding="utf-8") as f:
        target_df.to_csv(f, header=f.tell()==0, index = False, line_terminator = '\n')

In [13]:
# import csv
# output_csv = "{}{}_{}.csv".format(output_csv_path, race_code, output_csv_name_base)

# # We want to match the order of columns if we are appending new entries to an existing file.
# ### Because we are using a dictionary to capture the race results, and of the way dictionaries are ordered,
# ### It is possible for different scraping runs to attempt to order the columns differently.
# ### That would be confusing, so we force whatever order was established in the first run.
# ### (The bottom line is that it would really be a good idea to use a database here instead.)

# # Try to read the header from the existing file. If they exist, match the column order.
# try:
#     with open(output_csv, 'r', encoding="utf-8") as f:
#         reader = csv.reader(f)
#         header_list = next(reader)
#         # It is possible to be missing columns, so check to see if we have everything.
#         for x in header_list:
#             if x not in finishers_df.columns:
#                 finishers_df[x] = ''
#         finishers_df = finishers_df[header_list]
# # If we weren't able to open the file, we are going to create one from scratch.
# except:
#     print(f"{output_csv} does not exist, so I'll create it!")

# # This either creates the file, or appends to it without headers. We already took steps to make sure
# # the columns in finishers_df match the headers (if they exist).
# with open(output_csv, 'a', encoding="utf-8") as f:
#     finishers_df.to_csv(f, header=f.tell()==0, index = False, line_terminator = '\n')

In [14]:
# define the path and name for the output file
output_csv = "{}{}_{}.csv".format(output_csv_path, race_code, output_csv_name_base)

df_to_csv_writer(finishers_df, output_csv)

### Write scraping log to csv

In [15]:
log_df = pd.DataFrame(scraping_log)
log_csv = "{}{}_{}.csv".format(log_csv_path, race_code, log_csv_name_base)

# if the file does not exist, create it.
# if the file does exist, append to it without headers.
with open(log_csv, 'a', encoding="utf-8") as f:
    log_df.to_csv(f, header=f.tell()==0, line_terminator = '\n')